In [15]:
import pickle
import os
import numpy as np
import pandas as pd
import json

In [2]:
with open('../../genIII_data/JLab/Beam/Beam1_0-Cal.pkl', 'rb') as file:
    # Load the data from the pickle file
    din = pickle.load(file)
e_cut = 0.3

In [5]:
evt_energy = []
maxE = 0

for evt_idx, event in enumerate(din):
    energy = 0
    for e, x, y, z in event:
        if e > 0.3:
            if e > maxE:
                maxE = e
            energy += e
    evt_energy.append(energy)

In [29]:
# Generate the header section
header = {
    "version": "2.2.0",
    "experiment": f"ZDC Prototype",
    "energy_unit": "MIP", # Select "eV", "KeV", "MeV", "GeV", or "TeV"
    "color_bar": "Log", # Select "Lin" or "Log"
    "length_unit": "cm",
    "time_scale": 10,
    "particles": [
        {
        "angle_rad": [0.0 , 0.0],       
        "size": float(10),
        "color_rgba": [float(1), float(0), float(0), float(1)]
        }
    ],
    "tracker_settings": {
        "B_field_T": float(0),
        "tracker_boundary": [float(1), float(1), float(1)]
    }
}

# Main data structure
data = {
    "header": header,
    "events": []
}

In [30]:
indices = [i for i, energy in enumerate(evt_energy) if energy > 50]
events = []

for i in indices[0:100]:
    
    info_text = f"Event #{i}"
    event = {
        "event_data": {
            "info_text": info_text,
            "energy_scale": [0.3, maxE]
        },
        "hits": [],
        "clusters": [],
        "tracks": [],
        "jets": [],
        "blocks":[]
    }
    blocks = []
    for j in range(len(din[i])):
        
        energy = din[i][j][0]
        x = din[i][j][1]
        y = din[i][j][2]
        z = din[i][j][3]
            
        normalized_energy = np.log(energy / 0.3) / np.log(maxE / 0.3) if energy > 0.3 else 0

        red, green, blue, alpha = normalized_energy, 0, 1-normalized_energy, normalized_energy

        c = 299.792
        time = z / c
            
        block = {
                "position":[float(x), float(y), float(z)],
                "time_ns": float(time),
                "size": [float(48.8), float(48.8), float(4)],
                "color_rgba": [float(red), float(green), float(blue), float(alpha)]
            }
        blocks.append(block)
        
    event["blocks"] = blocks
    
        
    events.append(event)
    
data["events"] = events

In [31]:
eventPath = f'ZDCPrototype.json'
with open(eventPath, 'w') as outfile:
    json.dump(data, outfile, indent=4)